# Importation of needed libraries 

In [153]:
import pandas as pd 
import requests
import string
import seaborn as sns 

import pymysql
from sqlalchemy import create_engine
import getpass  # To get the password without showing the input

# Data Collection

In [154]:
# Set up the API endpoint and API key
base_url = "https://thecocktaildb.com/api/json/v2/"
api_key = "9973533"

In [155]:
#Use the endpoint to get the data of the most popular drinks in thecocktaildb 
endpoint = f"https://www.thecocktaildb.com/api/json/v2/{api_key}/popular.php"

response = requests.get(endpoint)
drinks_data = response.json()

drinks = drinks_data["drinks"]


In [156]:
#Create the dataframe for the popular drinks using json_normalize
popular_drinks_df = pd.json_normalize(drinks_data['drinks'])

In [157]:
#The base url for searching for every cocktail by letter
base_url = "https://www.thecocktaildb.com/api/json/v1/1/search.php?f="

#empty list to put in the data of each cocktail 
all_cocktails = []

# Loop through each letter of the alphabet
for letter in string.ascii_uppercase:
    # Construct the API endpoint for each letter
    endpoint = base_url + letter
    
    # Send GET request
    response = requests.get(endpoint)
    
    # Get the JSON response
    data = response.json()
    
    # Process the data
    if data['drinks']:
        for cocktail in data['drinks']:
            all_cocktails.append(cocktail)

In [161]:
#Create the dataframe for all the cocktails using json_normalize
all_cocktails_df = pd.json_normalize(all_cocktails)

# Data Wrangling 

In [162]:
popular_drinks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 51 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   idDrink                      20 non-null     object
 1   strDrink                     20 non-null     object
 2   strDrinkAlternate            0 non-null      object
 3   strTags                      16 non-null     object
 4   strVideo                     5 non-null      object
 5   strCategory                  20 non-null     object
 6   strIBA                       9 non-null      object
 7   strAlcoholic                 20 non-null     object
 8   strGlass                     20 non-null     object
 9   strInstructions              20 non-null     object
 10  strInstructionsES            9 non-null      object
 11  strInstructionsDE            19 non-null     object
 12  strInstructionsFR            0 non-null      object
 13  strInstructionsIT            20 non-n

In [163]:
#Verify the amount of rows and columns we have 
popular_drinks_df.shape

(20, 51)

In [164]:
#Check for duplicates
popular_drinks_df.duplicated().sum()

0

In [165]:
#Check for empty values 
popular_drinks_df.isna().sum()

idDrink                         0
strDrink                        0
strDrinkAlternate              20
strTags                         4
strVideo                       15
strCategory                     0
strIBA                         11
strAlcoholic                    0
strGlass                        0
strInstructions                 0
strInstructionsES              11
strInstructionsDE               1
strInstructionsFR              20
strInstructionsIT               0
strInstructionsZH-HANS         20
strInstructionsZH-HANT         20
strDrinkThumb                   0
strIngredient1                  0
strIngredient2                  0
strIngredient3                  0
strIngredient4                  8
strIngredient5                 15
strIngredient6                 18
strIngredient7                 20
strIngredient8                 20
strIngredient9                 20
strIngredient10                20
strIngredient11                20
strIngredient12                20
strIngredient1

In [166]:
#Drop all the columns that have empty values 
popular_drinks_df= popular_drinks_df.dropna(axis=1,how='all')

In [167]:
#Dropped 22 columns 
popular_drinks_df.shape

(20, 29)

In [168]:
#Check the names of the columns to see which ones we should keep for our analysis
popular_drinks_df.columns

Index(['idDrink', 'strDrink', 'strTags', 'strVideo', 'strCategory', 'strIBA',
       'strAlcoholic', 'strGlass', 'strInstructions', 'strInstructionsES',
       'strInstructionsDE', 'strInstructionsIT', 'strDrinkThumb',
       'strIngredient1', 'strIngredient2', 'strIngredient3', 'strIngredient4',
       'strIngredient5', 'strIngredient6', 'strMeasure1', 'strMeasure2',
       'strMeasure3', 'strMeasure4', 'strMeasure5', 'strMeasure6',
       'strImageSource', 'strImageAttribution', 'strCreativeCommonsConfirmed',
       'dateModified'],
      dtype='object')

In [169]:
# Drop the unnecessary columns
popular_drinks_df = popular_drinks_df.drop(['strInstructionsES','strInstructionsDE', 'strInstructionsIT','strDrinkThumb','strVideo','strImageSource', 'strImageAttribution', 'strCreativeCommonsConfirmed',
       'dateModified'], axis=1)

In [170]:
#Dropped 9 columns
popular_drinks_df.shape

(20, 20)

In [171]:
popular_drinks_df

,idDrink,strDrink,strTags,strCategory,strIBA,strAlcoholic,strGlass,strInstructions,strIngredient1,strIngredient2,strIngredient3,strIngredient4,strIngredient5,strIngredient6,strMeasure1,strMeasure2,strMeasure3,strMeasure4,strMeasure5,strMeasure6
0,11000,Mojito,"IBA,ContemporaryClassic,Alcoholic,USA,Asia,Veg...",Cocktail,Contemporary Classics,Alcoholic,Highball glass,Muddle mint leaves with sugar and lime juice. ...,Light rum,Lime,Sugar,Mint,Soda water,None,2-3 oz,Juice of 1,2 tsp,2-4,None,None
1,11001,Old Fashioned,"IBA,Classic,Alcoholic,Expensive,Savory",Cocktail,Unforgettables,Alcoholic,Old-fashioned glass,Place sugar cube in old fashioned glass and sa...,Bourbon,Angostura bitters,Sugar,Water,None,None,4.5 cL,2 dashes,1 cube,dash,None,None
2,11002,Long Island Tea,"Strong,Asia,StrongFlavor,Brunch,Vegetarian,Sour",Ordinary Drink,None,Alcoholic,Highball glass,Combine all ingredients (except cola) and pour...,Vodka,Light rum,Gin,Tequila,Lemon,Coca-Cola,1/2 oz,1/2 oz,1/2 oz,1/2 oz,Juice of 1/2,1 splash
3,11003,Negroni,"IBA,Classic",Ordinary Drink,Unforgettables,Alcoholic,Old-fashioned glass,"Stir into glass over ice, garnish and serve.",Gin,Campari,Sweet Vermouth,None,None,None,1 oz,1 oz,1 oz,None,None,None
4,11004,Whiskey Sour,"IBA,Classic,Alcoholic,ContemporaryClassic",Ordinary Drink,Unforgettables,Alcoholic,Old-fashioned glass,"Shake with ice. Strain into chilled glass, gar...",Blended whiskey,Lemon,Powdered sugar,Cherry,Lemon,None,2 oz,Juice of 1/2,1/2 tsp,1,1/2 slice,None
5,11005,Dry Martini,"IBA,Classic,Christmas,Alcoholic",Cocktail,Unforgettables,Alcoholic,Cocktail glass,Straight: Pour all ingredients into mixing gla...,Gin,Dry Vermouth,Olive,None,None,None,1 2/3 oz,1/3 oz,1,None,None,None
6,11006,Daiquiri,"IBA,Classic,Beach",Ordinary Drink,None,Alcoholic,Cocktail glass,Pour all ingredients into shaker with ice cube...,Light rum,Lime,Powdered sugar,None,None,None,1 1/2 oz,Juice of 1/2,1 tsp,None,None,None
7,11007,Margarita,"IBA,ContemporaryClassic",Ordinary Drink,Contemporary Classics,Alcoholic,Cocktail glass,Rub the rim of the glass with the lime slice t...,Tequila,Triple sec,Lime juice,Salt,None,None,1 1/2 oz,1/2 oz,1 oz,None,None,None
8,11008,Manhattan,"IBA,Classic,Alcoholic",Cocktail,Unforgettables,Alcoholic,Cocktail glass,"Stirred over ice, strained into a chilled glas...",Sweet Vermouth,Bourbon,Angostura bitters,Ice,Maraschino cherry,Orange peel,3/4 oz,2 1/2 oz Blended,dash,2 or 3,1,1 twist of
9,11009,Moscow Mule,"IBA,ContemporaryClassic",Punch / Party Drink,Contemporary Classics,Alcoholic,Copper Mug,Combine vodka and ginger beer in a highball gl...,Vodka,Lime juice,Ginger ale,None,None,None,2 oz,2 oz,8 oz,None,None,None


In [172]:
#Verify the amount of rows and columns we have
all_cocktails_df.shape

(426, 51)

In [173]:
#Check for duplicates
all_cocktails_df.duplicated().sum()

0

In [174]:
#Check for null values 
all_cocktails_df.isna().sum()

idDrink                          0
strDrink                         0
strDrinkAlternate              426
strTags                        325
strVideo                       404
strCategory                      0
strIBA                         365
strAlcoholic                     0
strGlass                         0
strInstructions                  0
strInstructionsES              401
strInstructionsDE               43
strInstructionsFR              426
strInstructionsIT               10
strInstructionsZH-HANS         426
strInstructionsZH-HANT         426
strDrinkThumb                    0
strIngredient1                   0
strIngredient2                   0
strIngredient3                  64
strIngredient4                 194
strIngredient5                 283
strIngredient6                 362
strIngredient7                 407
strIngredient8                 422
strIngredient9                 424
strIngredient10                425
strIngredient11                425
strIngredient12     

In [175]:
#Drop completely empty columns 
all_cocktails_df = all_cocktails_df.dropna(axis=1,how='all')

In [176]:
#Dropped 11 columns 
all_cocktails_df.shape

(426, 40)

In [177]:
#Verify the names of the columns that are not needed 
all_cocktails_df.columns

Index(['idDrink', 'strDrink', 'strTags', 'strVideo', 'strCategory', 'strIBA',
       'strAlcoholic', 'strGlass', 'strInstructions', 'strInstructionsES',
       'strInstructionsDE', 'strInstructionsIT', 'strDrinkThumb',
       'strIngredient1', 'strIngredient2', 'strIngredient3', 'strIngredient4',
       'strIngredient5', 'strIngredient6', 'strIngredient7', 'strIngredient8',
       'strIngredient9', 'strIngredient10', 'strIngredient11', 'strMeasure1',
       'strMeasure2', 'strMeasure3', 'strMeasure4', 'strMeasure5',
       'strMeasure6', 'strMeasure7', 'strMeasure8', 'strMeasure9',
       'strMeasure10', 'strMeasure11', 'strMeasure12', 'strImageSource',
       'strImageAttribution', 'strCreativeCommonsConfirmed', 'dateModified'],
      dtype='object')

In [178]:
#Drop unnecessary columns 
all_cocktails_df = all_cocktails_df.drop(['strInstructionsES','strInstructionsDE', 'strInstructionsIT','strDrinkThumb','strVideo','strImageSource', 'strImageAttribution', 'strCreativeCommonsConfirmed',
       'dateModified'], axis=1)

In [179]:
#Dropped 9 columns 
all_cocktails_df.shape

(426, 31)

In [180]:
all_cocktails_df

,idDrink,strDrink,strTags,strCategory,strIBA,strAlcoholic,strGlass,strInstructions,strIngredient1,strIngredient2,...,strMeasure3,strMeasure4,strMeasure5,strMeasure6,strMeasure7,strMeasure8,strMeasure9,strMeasure10,strMeasure11,strMeasure12
0,17222,A1,None,Cocktail,None,Alcoholic,Cocktail glass,"Pour all ingredients into a cocktail shaker, m...",Gin,Grand Marnier,...,1/4 Shot,1/8 Shot,None,None,None,None,None,None,None,None
1,13501,ABC,None,Shot,None,Alcoholic,Shot glass,Layered in a shot glass.,Amaretto,Baileys irish cream,...,1/3,None,None,None,None,None,None,None,None,None
2,17225,Ace,None,Cocktail,None,Alcoholic,Martini Glass,Shake all the ingredients in a cocktail shaker...,Gin,Grenadine,...,1/2 shot,1/2 shot,1/2 Fresh,None,None,None,None,None,None,None
3,14610,ACID,None,Shot,None,Alcoholic,Shot glass,Poor in the 151 first followed by the 101 serv...,151 proof rum,Wild Turkey,...,None,None,None,None,None,None,None,None,None,None
4,17837,Adam,"Alcoholic,Holiday",Ordinary Drink,None,Alcoholic,Cocktail glass,"In a shaker half-filled with ice cubes, combin...",Dark rum,Lemon juice,...,1 tsp,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,17027,Zima Blaster,None,Ordinary Drink,None,Alcoholic,Hurricane glass,"Fill glass with ice. Pour in Chambord, then fi...",Zima,Chambord raspberry liqueur,...,None,None,None,None,None,None,None,None,None,None
422,14594,Zizi Coin-coin,None,Punch / Party Drink,None,Alcoholic,Margarita/Coupette glass,"Pour 5cl of Cointreau on ice, add 2cl of fresh...",Cointreau,Lemon juice,...,cubes,or lime\n,None,None,None,None,None,None,None,None
423,15801,Zimadori Zinger,None,Punch / Party Drink,None,Alcoholic,Collins glass,Pour Zima in a collins glass over ice and then...,Midori melon liqueur,Zima,...,None,None,None,None,None,None,None,None,None,None
424,14065,Zippy's Revenge,None,Cocktail,None,Alcoholic,Old-fashioned glass,Mix Kool-Aid to taste then add Rum and ammaret...,Amaretto,Rum,...,4 oz Grape,None,None,None,None,None,None,None,None,None


# Feature Engineering

In [181]:
popular_drinks_df.shape

(20, 20)

In [182]:
popular_drinks_df.describe()

,idDrink,strDrink,strTags,strCategory,strIBA,strAlcoholic,strGlass,strInstructions,strIngredient1,strIngredient2,strIngredient3,strIngredient4,strIngredient5,strIngredient6,strMeasure1,strMeasure2,strMeasure3,strMeasure4,strMeasure5,strMeasure6
count,20,20,16,20,9,20,20,20,20,20,20,12,5,2,20,20,20,9,4,2
unique,20,20,15,3,2,1,5,20,14,13,16,10,4,2,9,12,14,8,4,2
top,11000,Mojito,"IBA,ContemporaryClassic",Ordinary Drink,Unforgettables,Alcoholic,Cocktail glass,Muddle mint leaves with sugar and lime juice. ...,Gin,Triple sec,Sugar,Lemon juice,Lemon,Coca-Cola,1 oz,1 oz,1 oz,1,Juice of 1/2,1 splash
freq,1,1,2,15,6,20,12,1,3,3,2,2,2,1,7,7,4,2,1,1


In [148]:
popular_drinks_df['strIngredient1'].value_counts()

strIngredient1
Gin                 3
Light rum           2
Vodka               2
Blended whiskey     2
Dry Vermouth        2
Bourbon             1
Tequila             1
Sweet Vermouth      1
Apricot brandy      1
Triple sec          1
Southern Comfort    1
Orange bitters      1
Brandy              1
Lemon vodka         1
Name: count, dtype: int64

# Statistical Analysis

# Data Visualization 

Sending the cleaned data to MySQL for storage and to be later used in Tableau. 

In [ ]:
password = getpass.getpass()

In [184]:
pip install python-dotenv

In [ ]:
bd = "final_project"
connection_string = 'mysql+pymysql://root:' + password + '@localhost/'+bd
engine = create_engine(connection_string)
engine

In [ ]:
popular_drinks_df.to_sql('popular_drinks_df', con=engine, if_exists='replace', index=False)

In [ ]:
all_cocktails_df.to_sql('all_cocktails_df', con=engine, if_exists='replace', index=False)
